### Install the following libraries if required

In [ ]:
!pip install pystan~=2.14
!pip install cmdstanpy>=1.0.4
!pip install fbprophet

### Standard Imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Prophet Imports

In [6]:
from prophet import Prophet
from prophet.plot import plot_plotly
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
from prophet.plot import plot_cross_validation_metric

### Statsmodels Imports

In [7]:
import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

### Define Functions

In [8]:
def transform_to_monthly(df):
    df1 = df.copy()
    df1['ds'] = pd.to_datetime(df1['ds'])
    # Getting month number
    df1['Month'] = df1['ds'].dt.month
    df1['Year'] = df1['ds'].dt.year
    #print(df1)
    # Grouping based on required values
    df1 = df1.groupby(['Year','Month']).agg({'ds': 'min', 'y':'mean'})
    #print(df1)
    # Reset the index
    df1.reset_index(inplace=True)
    # Drop Year and Week
    df1.drop(['Year', 'Month'], axis=1, inplace=True)
    df1['ds'] = pd.to_datetime(df1['ds']) + MonthEnd(1)
    #print(df1)
    df1 = df1.set_index('ds')
    return df1

In [9]:
def run_seasonal_decompose(df1):
    df1.plot(x='ds', y='y', kind='line', grid=1)
    plt.pyplot.show()
    df = transform_to_monthly(df1)
    decompose_result = seasonal_decompose(df, model='multiplicative')
    decompose_result.plot().show()
    decompose_multi_resid = decompose_result.resid.sum()
    decompose_result = seasonal_decompose(df, model='additive')
    decompose_result.plot().show()
    decompose_add_resid = decompose_result.resid.sum()
    if decompose_multi_resid < decompose_add_resid:
        print("Multiplicate  Model")
    else:
        print("Additive  Model")

In [11]:
def run_forecasting(params):
    df_train = params['train']
    df_test = params['test']
    df_test['ds'] = pd.to_datetime(df_test['ds'])
    #print(df_test)
    model = Prophet()
    model.fit(df_train)
    # Predict
    df_future = model.make_future_dataframe(periods=params['forecast_period'])
    forecast = model.predict(df_future)
    return (model, forecast)

In [34]:
def run_adfuller(df, name):
    test_statistic, pvalues, numusedlag, numobs, critical_vals, icbest =  adfuller(df)
    print('p-values:', pvalues)
    print('Test statistic:', test_statistic)
    print('1% critical values:', critical_vals['1%'])
    print('5% critical values:', critical_vals['5%'])
    print('10% critical values:', critical_vals['10%'])
    if (pvalues > 0.05) & \
        (test_statistic > critical_vals['1%']) & \
        (test_statistic > critical_vals['5%']) & \
        (test_statistic > critical_vals['10%']):
        print(f"{name} is a non-stationary time series")
    else:
        print(f"{name} is a stationary time series")
    print('------------------------------------------------')

In [22]:
def run_cv(df, hor):
    df_cv = cross_validation(model, horizon=hor)
    df_performance = performance_metrics(df_cv)
    selected_horizon = df_performance[df_performance['horizon'] == hor]
    print(df_performance['mape'].max())
    print(df_performance['mape'].min())
    fig1 = plot_cross_validation_metric(df_cv, metric='mape')

### Read and Process the Time Series Data

In [154]:
your_drive_path = "/content/drive/MyDrive/Colab Notebooks/dataset/timeseries/"
##
selected_timeseries = []
##
resp_admission = pd.read_csv(your_drive_path+'respiratory_admission.csv', index_col=[0])
resp_admission.reset_index(inplace=True)
train = resp_admission
test = resp_admission[resp_admission['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Respiratory Admission'})
##
pypi_download = pd.read_csv(your_drive_path+'pypi_prophet_download.csv', index_col=[0])
pypi_download.reset_index(inplace=True)
pypi_download.columns = ['ds', 'y']
train = pypi_download
test = pypi_download[pypi_download['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Pypi Prophet Download'})
##
peyton_manning = pd.read_csv(your_drive_path+'peyton_manning.csv', index_col=[0])
peyton_manning.reset_index(inplace=True)
train = peyton_manning
test = peyton_manning[peyton_manning['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Peyton Manning Wiki'})
##
retail_sales = pd.read_csv(your_drive_path+'retail_sales.csv', index_col = 0)
retail_sales.reset_index(inplace=True)
retail_sales.rename(columns={'month':'ds', 'sales_total':'y'}, inplace=True)
retail_sales['ds'] = pd.to_datetime(retail_sales['ds'])
train = retail_sales
test = retail_sales[retail_sales['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Retail Sales'})
##
bitcoin_prices = pd.read_csv(your_drive_path+'bitcoin_prices.csv', index_col=[0])
bitcoin_prices = bitcoin_prices.filter(['ds', 'y'])
bitcoin_prices['ds'] = pd.to_datetime(bitcoin_prices['ds'])
train = bitcoin_prices
test = bitcoin_prices[bitcoin_prices['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Bitcoin Prices'})
##
broiler_prices = pd.read_csv(your_drive_path+'broiler_prices.csv')
train = broiler_prices
test = broiler_prices[broiler_prices['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Broiler Prices'})
##
milkcow_prices = pd.read_csv(your_drive_path+'milkcow_prices.csv')
milkcow_prices['ds'] = pd.to_datetime(milkcow_prices['ds'])
train = milkcow_prices
test = milkcow_prices[milkcow_prices['ds']>'2018-02-28']
selected_timeseries.append({'train_df': train, 'test_df':test, 'name': 'Milk Cow Prices'})

### Run the Statistical Hypothesis Test

In [ ]:
for i in range(len(selected_timeseries)):
  run_adfuller(selected_timeseries[i]['train_df'].y.values, selected_timeseries[i]['name'])

### Decompose the Time Series Data

In [ ]:
for i in range(len(selected_timeseries)):
    df = selected_timeseries[i]
    df_train = df['train_df']
    df_test = df['test_df']
    model_params = {'train': df_train, 'test': df_test, 'forecast_period': 120}
    model, forecast = run_forecasting(model_params)
    run_cv(model, '120 days')

### Test the Forecasting Model of the Broiler Prices Time Series 

In [ ]:
train = broiler_prices[broiler_prices['ds']<='2020-10-31']
test = broiler_prices[broiler_prices['ds']>'2020-10-31']
model_params = {'train': train, 'test': test, 'forecast_period': 120}
model, forecast = run_forecasting(model_params)
fig = model.plot(forecast)
ax = fig.gca()
ax.plot(test["ds"], test["y"], 'r.')